# DATA 202 @ Calvin University, FA19

## Lab 2: More Pandas!

**Author**: Kenneth C. Arnold

**Adapted with permission from Berkeley DATA 100 Summer 2019**

## Some useful resources


Introductory:

* [Getting started with Python for research](https://github.com/TiesdeKok/LearnPythonforResearch), a gentle introduction to Python in data-intensive research.

* [A Whirlwind Tour of Python](https://jakevdp.github.io/WhirlwindTourOfPython/index.html), by Jake VanderPlas, another quick Python intro (with notebooks).

Core Pandas/Data Science books:

* [The Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/), by Jake VanderPlas.

* [Python for Data Analysis, 2nd Edition](http://proquest.safaribooksonline.com/book/programming/python/9781491957653), by  Wes McKinney, creator of Pandas. [Companion Notebooks](https://github.com/wesm/pydata-book)

* [Effective Pandas](https://github.com/TomAugspurger/effective-pandas), a book by Tom Augspurger, core Pandas developer.


Complementary resources:

* [An introduction to "Data Science"](https://github.com/stefanv/ds_intro), a collection of Notebooks by BIDS' [Stéfan Van der Walt](https://bids.berkeley.edu/people/st%C3%A9fan-van-der-walt).

* [Effective Computation in Physics](http://proquest.safaribooksonline.com/book/physics/9781491901564), by Kathryn D. Huff; Anthony Scopatz. [Notebooks to accompany the book](https://github.com/physics-codes/seminar). Don't be fooled by the title, it's a great book on modern computational practices with very little that's physics-specific.


OK, let's load and configure some of our core libraries (as an aside, you can find a nice visual gallery of available matplotlib styles [here](https://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html)).

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

sns.set()
sns.set_context('notebook')
np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.options.display.max_rows = 10
pd.options.display.max_columns = 8
pd.set_option('precision', 2)
# This option stops scientific notation for pandas
# pd.set_option('display.float_format', '{:.2f}'.format)

## Reading in DataFrames from Files

Pandas has a number of very useful file reading tools. You can see them enumerated by typing "pd.re" and pressing tab. We'll be using read_csv today. 

In [2]:
elections = pd.read_csv("elections.csv")
elections # if we end a cell with an expression or variable name, the result will print

## Basic Inspection
You'll probably want to do examine just about every DataFrame you work with using methods like this:

In [3]:
elections.info()

In [4]:
elections.head(n=7)  # What is the default value of n?

There is also a tail command.

In [5]:
elections.tail(7)

Note that `describe` by default only looks at the numeric columns.

In [6]:
elections.describe()

How many different unique years were there?

In [7]:
# Your code here:


In [8]:
# Your code here


## Exercise: inspect a different DataFrame

In [9]:
mottos = pd.read_csv("mottos.csv", index_col = "State")
mottos.head(5)

In [10]:
# Your code here


In [11]:
# Your code here


## The [] Operator

**Ground rule**: Before running one of the following cells, tell your neighbor what you think the result will be.

The DataFrame class has an indexing operator `[]` that lets you do a variety of different things.

Getting a column by name:

In [12]:
elections["Candidate"].head(6)

If you actually wanted a one-column DataFrame instead of a Series:

In [13]:
elections["Candidate"].to_frame()

The [] operator also accepts a list of strings:

In [14]:
elections[["Candidate", "Party"]].head(6)

**Exercise**: How many times did each Party run?

In [15]:
# your code here


## Boolean Array Selection

The `[]` operator also supports array of booleans as an input. In this case, the array must be exactly as long as the number of rows. The result is a filtered version of the data frame, where only rows corresponding to True appear.

In [16]:
elections[[False, False, False, False, False, 
          False, False, True, False, False,
          True, False, False, False, True,
          False, False, False, False, False,
          False, False, True]]

One very common task in Data Science is filtering. Boolean Array Selection is one way to achieve this in Pandas. We start by observing logical operators like the equality operator can be applied to Pandas Series data to generate a Boolean Array. For example, we can compare the 'Result' column to the String 'win':

In [17]:
elections.head(5)

In [18]:
iswin = elections['Result'] == 'win'
iswin.head(5)

The output of the logical operator applied to the Series is another Series with the same name and index, but of datatype boolean.

In [19]:
iswin.dtype

In [20]:
elections[iswin]

Above, we've assigned the result of the logical operator to a new variable called `iswin`. This is uncommon. Usually, the series is created and used on the same line. Such code is a little tricky to read at first, but you'll get used to it quickly.

In [21]:
elections[elections['Result'] == 'win']

**Exercise**: Get the data for all Independent candidates

In [22]:
# Your code here


We can select multiple criteria by creating multiple boolean Series and combining them using the `&` operator.

In [23]:
elections[(elections['Result'] == 'win') & (elections['%'] < 50)]

**Exercise**: What state has a Spanish motto?

In [24]:
# Your code here


**Exercise**: How many states have non-English mottos?

In [25]:
# Your code here


## `loc` and `iloc`

To go beyond what we can do with `[]` (getting named columns and boolean subsets of rows), we'll need to use more general accessor methods. The most common are `loc` and `iloc`:

* `loc` uses the **names** (the row and column indices)
* `iloc` ignores the indices and looks up by **place**.

In [26]:
elections["Party"].head()

In [27]:
elections.loc[:, "Party"].head()

In [28]:
mottos.loc["Michigan"]

In [29]:
mottos.loc["Michigan", "Translation"]

In [30]:
mottos.iloc[10]

In [31]:
mottos.iloc[10, 1]

In [32]:
elections.iloc[0]

# Utility functions

There are also a ton of useful utility methods we can use with Data Frames and Series. For example, we can create a copy of a data frame sorted by a specific column using `sort_values`.

In [33]:
elections.sort_values('%')

In [34]:
elections.sort_values('%').iloc[-4:]

As mentioned before, all Data Frame methods return a copy and do **not** modify the original data structure, unless you set inplace to True.

In [35]:
elections.head(5)

If we want to sort in reverse order, we can set `ascending=False`.

In [36]:
elections.sort_values('%', ascending=False)

For Series, the `value_counts` method is often quite handy.

## Case Study: What was the most popular name in Michigan last year?

Start by downloading the baby name data from the Social Security Administration.

* [Direct link to ZIP file](https://www.ssa.gov/oact/babynames/state/namesbystate.zip)
* https://www.ssa.gov/OACT/babynames/index.html
* https://www.ssa.gov/data

Extract it to the current directory. Here's a block of code that does all of that in Python; you don't need to understand how it works.

In [37]:
# Download the ZIP file
import requests
from pathlib import Path

namesbystate_path = Path('namesbystate.zip')
data_url = 'https://www.ssa.gov/oact/babynames/state/namesbystate.zip'

if not namesbystate_path.exists():
    print('Downloading...', end=' ')
    resp = requests.get(data_url)
    with namesbystate_path.open('wb') as f:
        f.write(resp.content)
    print('Done!')
else:
    print("File already downloaded.")

In [38]:
# Extract the data.
import zipfile
zf = zipfile.ZipFile(namesbystate_path, 'r')
zf.extractall(path="names_by_state")
zf.close()

Let's have a look at the Michigan data, it should give us an idea about the structure of the whole thing:

In [39]:
mi_filename = "names_by_state/MI.TXT"
with open(mi_filename) as f:
    for i in range(10):
        print(f.readline().rstrip())

This is equivalent (on macOS or Linux) to extracting the full `MI.TXT` file to disk and then using the `head` command (if you're on Windows, don't try to run the cell below):

In [40]:
!head {mi_filename}

A couple of practical comments:

* The above is using special tricks in Jupyter's "IPython" kernels that let you call operating system commands via `!cmd`, and that expand Python variables in such commands with the `{var}` syntax. You can find more about IPython's special tricks [in this tutorial](https://github.com/ipython/ipython-in-depth/blob/master/examples/IPython%20Kernel/Beyond%20Plain%20Python.ipynb).

* `head` doesn't work on Windows, though there are equivalent Windows commands. But by using Python code, even if it's a little bit more verbose, we have a 100% portable solution.

## Load the data

In [41]:
def load_names(state):
    field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
    return pd.read_csv(f'names_by_state/{state}.TXT', header=None, names=field_names)
mi_names = load_names(state='MI')

In [42]:
mi_names.head()

## Exploration review

Answer the following questions.

How many rows are in the dataset? How many columns?

In [43]:
# your code here


*Your answer here*


What is the range of years contained in this dataset?

In [44]:
# your code here


How many different names are there?

In [45]:
# your code here


### Indexing Review

Let's play around a bit with our indexing techniques from earlier today.

**Try the following**:

Extract the first few entries in the Name column, in 3 different ways:

1. Using `mi_names[]`

In [50]:
# Your code here


2. Using `mi_names.loc[]`

In [51]:
# Your code here


3. Using `mi_names.iloc[]`

In [53]:
# Your code here


### Sorting

What we've done so far is NOT exploratory data analysis. We were just playing around a bit with the capabilities of the pandas library. Now that we're done, let's turn to the problem at hand: Identifying the most common name in Michigan last year.

**Step 1**: Find the most recent year in the DataFrame and extract only the rows for that year.

In [54]:
# your code here


**Step 2**: Get the top-10 names by birth count.

In [55]:
# your code here


**Bonus**: break that down by sex. (you may find the `display` built-in function helpful.)

In [56]:
# your code here


**Bonus**: What is the meaning of this?

In [57]:
(mi_names_for_most_recent_year.Sex == "M").mean()